***GENERATED CODE FOR eligdenautoclassv1 PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import warnings
warnings.filterwarnings('ignore')


class RDBMSConnector:

    def fetch(spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        return spark.read.format("jdbc") \
            .option("url", f"jdbc:sqlserver://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option(eval(config)['qtype'], eval(config)['query']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()

    def put(df, spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        # Write modes: overwrite, append
        df.write.mode(eval(config)['writemode'])\
            .format('jdbc') \
            .option("url", f"jdbc:{eval(config)['dbtype']}://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option("dbtable", eval(config)['table']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .save()


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def ExtractDateTransform(df, params, transformationData={}):
    transform_params = params
    dfReturn = df
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: ''})
    dfReturn = dfReturn.withColumn(
        feature+'dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'year', year(col(feature)))
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ChargeDetailID', 'transformation_label': 'String Indexer'}], 'feature': 'ChargeDetailID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '22', 'mean': '', 'stddev': '', 'min': '17246A3B-4561-4F4A-A688-078711E2799D', 'max': 'F9640CD4-30FA-4366-A361-D714E737FABE', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ChargeDetailID'}, {'feature_label': 'ChargeDetailID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ChargeDetailID')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'DateofService', 'transformation_label': 'Extract Date'}], 'feature': 'DateofService', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'DateofService'}, {'feature_label': 'DateofService', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('DateofService')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'DateOfEntry', 'transformation_label': 'Extract Date'}], 'feature': 'DateOfEntry', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'DateOfEntry'}, {'feature_label': 'DateOfEntry', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('DateOfEntry')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'TransactionDate', 'transformation_label': 'Extract Date'}], 'feature': 'TransactionDate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'TransactionDate'}, {'feature_label': 'TransactionDate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('TransactionDate')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'ReasonDate', 'transformation_label': 'Extract Date'}], 'feature': 'ReasonDate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'ReasonDate'}, {'feature_label': 'ReasonDate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('ReasonDate')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'CloseDate', 'transformation_label': 'Extract Date'}], 'feature': 'CloseDate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'CloseDate'}, {'feature_label': 'CloseDate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('CloseDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'}], 'feature': 'CPTCode', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '22', 'mean': '88519.53', 'stddev': '9697.68', 'min': '67028', 'max': 'J2778PF5', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CPTCode'}, {'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CPTCode')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CptModifier', 'transformation_label': 'String Indexer'}], 'feature': 'CptModifier', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '22', 'mean': '19.15', 'stddev': '20.31', 'min': '0000', 'max': 'LT', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CptModifier'}, {'feature_label': 'CptModifier', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CptModifier')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'pdiagnosis1', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '22', 'mean': '', 'stddev': '', 'min': 'D31.32', 'max': 'H43.813', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis1'}, {'feature_label': 'pdiagnosis1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('pdiagnosis1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'pdiagnosis2', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '22', 'mean': '', 'stddev': '', 'min': 'D31.31', 'max': 'Z79.4', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis2'}, {'feature_label': 'pdiagnosis2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('pdiagnosis2')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '22', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'M', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Reasoncode', 'transformation_label': 'String Indexer'}], 'feature': 'Reasoncode', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '22', 'mean': '44.4', 'stddev': '59.72', 'min': '16', 'max': 'N362', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Reasoncode'}, {'feature_label': 'Reasoncode', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Reasoncode')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ReasonCategory', 'transformation_label': 'String Indexer'}], 'feature': 'ReasonCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '22', 'mean': '', 'stddev': '', 'min': 'Benefits Max/Exh', 'max': 'Timely Filing', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ReasonCategory'}, {'feature_label': 'ReasonCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ReasonCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ReasonDescription', 'transformation_label': 'String Indexer'}], 'feature': 'ReasonDescription', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '22', 'mean': '', 'stddev': '', 'min': 'Claim/service lacks information or has submission/billing error(s). Usage: Do not use this code for claims attachment(s)/other documentation. At least one Remark Code must be provided (may be comprised of either the NCPDP Reject Reason Code, or Remittance Advice Remark Code that is not an ALERT.) Refer to the 835 Healthcare Policy Identification Segment (loop 2110 Service Payment Information REF), if present.', 'max': 'This service/equipment/drug is not covered under the patient’s current benefit plan', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ReasonDescription'}, {'feature_label': 'ReasonDescription', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ReasonDescription')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'}], 'feature': 'DepartmentCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '22', 'mean': '', 'stddev': '', 'min': 'Cardiology', 'max': 'Pulmonary ', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DepartmentCategory'}, {'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DepartmentCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SpecialityCategory', 'transformation_label': 'String Indexer'}], 'feature': 'SpecialityCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '22', 'mean': '', 'stddev': '', 'min': 'Ophtamology', 'max': 'Ophtamology', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SpecialityCategory'}, {'feature_label': 'SpecialityCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SpecialityCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProviderID', 'transformation_label': 'String Indexer'}], 'feature': 'ProviderID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '22', 'mean': '', 'stddev': '', 'min': '03151B5D-685B-4211-B01F-7BBB1C4A8A49', 'max': 'CBBD4925-1757-45BC-A853-1C2CA203B9E3', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ProviderID'}, {'feature_label': 'ProviderID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ProviderID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PayerId', 'transformation_label': 'String Indexer'}], 'feature': 'PayerId', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '22', 'mean': '', 'stddev': '', 'min': '56BFE483-84DF-4FCD-BB0B-4E41BA841A35', 'max': 'FCE8E295-4222-43DC-9E1E-13222F1045C7', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PayerId'}, {'feature_label': 'PayerId', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PayerId')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SecondaryPayerID', 'transformation_label': 'String Indexer'}], 'feature': 'SecondaryPayerID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '22', 'mean': '', 'stddev': '', 'min': '0BC878EE-8F48-415E-9233-25A7C28264DB', 'max': 'EEAC7713-511B-4D72-A6F2-432DD651D92A', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SecondaryPayerID'}, {'feature_label': 'SecondaryPayerID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SecondaryPayerID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TertiaryPayerID', 'transformation_label': 'String Indexer'}], 'feature': 'TertiaryPayerID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '22', 'mean': '', 'stddev': '', 'min': '41B329D5-6CFA-464C-9977-0438E0F9B8CE', 'max': 'FF58D17E-F6AC-4390-999A-AE5378E83883', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'TertiaryPayerID'}, {'feature_label': 'TertiaryPayerID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('TertiaryPayerID')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run eligdenautoclassv1Hooks.ipynb
try:
	#sourcePreExecutionHook()

	sqlserver = RDBMSConnector.fetch(spark, "{'host': 'wsc-sqlserver1.database.windows.net', 'port': '1433', 'password': 'G5gTW5rnKVeAZWXm', 'database': 'Demodb2020', 'optionalDB': 'Demodb2020', 'user': 'Numtra', 'qtype': 'query', 'query': "SELECT * FROM\r\n(\r\n\t\t\tSELECT DISTINCT\r\n\t\t\tcd.ChargeDetailID, \r\n\t\t\tcd.DateofService, \r\n\t\t\tcd.DateOfEntry,\r\n\t\t\tcd.Amount,\r\n\t\t\tt.TransactionDate,\r\n\t\t\ttr.ReasonDate,\r\n\t\t\tt.CloseDate,\r\n\t\t\tcd.PatientBal, cd.InsuranceBal, cd.PrimaryInsuranceBal,\r\n\t\t\tcptc.CPTCode,\r\n\t\t\tCASE WHEN cd.CPTModifier IS NOT NULL THEN cd.CPTModifier ELSE '0000' END as CptModifier,\r\n\t\t\tdc.DiagnosisCode as pdiagnosis1,\r\n\t\t\tdc2.DiagnosisCode as pdiagnosis2,\r\n\t\t\tIsBillable=(SELECT Case WHEN e.IsBillable = 'true' THEN 1 ELSE 0 END), \r\n\t\t\tIsauthorized=(SELECT Case WHEN (apa.[AuthorizationCode]  IS NOT NULL) THEN 1 ELSE 0 END), \r\n\t\t\tPredeterminationIndicator=(SELECT Case WHEN (apa.[PredeterminationIndicator]  IS NOT NULL) THEN 1 ELSE 0 END), \r\n\t\t\tMedNecessityRequiredIndicator=(SELECT Case WHEN (apa.[MedNecessityRequiredIndicator]  IS NOT NULL) THEN 1 ELSE 0 END),\r\n\t\t\tFLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) AS Age, p.Gender,\r\n\t\t\tDATEDIFF(day, t.TransactionDate, tr.ReasonDate) as days_to_denial, \r\n\t\t\ttrc.Reasoncode,\r\n\t\t\ttrc.ReasonCategory,\r\n\t\t\ttrc.ReasonDescription,\r\n\t\t\td.DepartmentCategory,\r\n\t\t\ts.SpecialityCategory,\r\n\t\t\tpp.ProviderID,\r\n\t\t\tt.PayerId,t.SecondaryPayerID,t.TertiaryPayerID,\r\n\t\t\tlabel = 0 \r\n\t\t\t\r\n\t\t\tfrom ChargeDetail cd\r\n\t\t\tinner join Encounter e WITH (NOLOCK) on cd.DateOfService = e.DateOfService and cd.PatientID = e.PatientID\r\n\t\t\tleft outer join AuthorizationsPA apa WITH (NOLOCK) on e.PatientID = apa.PatientID and apa.PayerId = e.PayerID  \r\n\t\t\tinner join Transactions t WITH (NOLOCK) on cd.ChargeDetailID = t.ChargeDetailID\r\n\t\t\tinner join TransactionReason tr WITH (NOLOCK) on tr.ChargeDetailID = t.ChargeDetailID\r\n\t\t\tinner join TransactionReasonCode trc WITH (NOLOCK) on trc.TransactionReasonCodeID = tr.TransactionReasonCodeID\r\n\t\t\tinner join Patient p WITH (NOLOCK) on p.PatientID = cd.PatientID\r\n\t\t\tinner join Department d WITH (NOLOCK) on d.DepartmentID = cd.DepartmentID\r\n\t\t\tinner join Provider pp WITH (NOLOCK) on pp.ProviderID = cd.ProviderID\r\n\t\t\tinner join Speciality s WITH (NOLOCK) on s.SpecialityID = pp.SpecialityID\r\n\t\t\tinner join DiagnosisCode dc WITH (NOLOCK) on dc.DiagnosisCodeID = cd.DiagnosisCodeID1\r\n\t\t\tinner join DiagnosisCode dc2 WITH (NOLOCK) on dc2.DiagnosisCodeID = cd.DiagnosisCodeID2\r\n\t\t\tinner join CptCode cptc WITH (NOLOCK) on cptc.CPTCodeID = cd.CPTCodeID\r\n\t\t\tinner join Transactiontype tt on t.transactiontypeid = tt.TransactionTypeID\r\n\t\t\tWHERE\r\n\t\t\tcd.DateOfService between '1/01/2020' and '5/31/2020' \r\n\t\t\tand cd.Amount >.01 and t.CloseDate is not null\r\n\t\t\tand trc.reasonType = 'Denial'  \r\n\t\t\tand tt.TransactionTypeCategory = 'Charges'\r\n\t\t\tand trc.ReasonCode IN ('19','20','21','22','24','26','27','28','30','31','32')\r\n\r\nUNION\r\n\r\nSELECT DISTINCT\r\n\r\n\tcd.ChargeDetailID, \r\n\tcd.DateofService, \r\n\tcd.DateOfEntry,\r\n\tcd.Amount,\r\n\tt.TransactionDate,\r\n\ttr.ReasonDate,\r\n\tt.CloseDate,\r\n\tcd.PatientBal, cd.InsuranceBal, cd.PrimaryInsuranceBal,\r\n\tcptc.CPTCode,\r\n\tCASE WHEN cd.CPTModifier IS NOT NULL THEN cd.CPTModifier ELSE '0000' END as CptModifier,\r\n\tdc.DiagnosisCode as pdiagnosis1,\r\n\tdc2.DiagnosisCode as pdiagnosis2,\r\n\tIsBillable=(SELECT Case WHEN e.IsBillable = 'true' THEN 1 ELSE 0 END), \r\n\tIsauthorized=(SELECT Case WHEN (apa.[AuthorizationCode]  IS NOT NULL) THEN 1 ELSE 0 END), \r\n\tPredeterminationIndicator=(SELECT Case WHEN (apa.[PredeterminationIndicator]  IS NOT NULL) THEN 1 ELSE 0 END), \r\n\tMedNecessityRequiredIndicator=(SELECT Case WHEN (apa.[MedNecessityRequiredIndicator]  IS NOT NULL) THEN 1 ELSE 0 END),\r\n\tFLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) AS Age, p.Gender,\r\n\tDATEDIFF(day, t.TransactionDate, tr.ReasonDate) as days_to_denial, \r\n\ttrc.Reasoncode,\r\n\ttrc.ReasonCategory,\r\n\ttrc.ReasonDescription,\r\n\td.DepartmentCategory,\r\n\ts.SpecialityCategory,\r\n\tpp.ProviderID,\r\n\tt.PayerId,t.SecondaryPayerID,t.TertiaryPayerID,\r\n\tlabel = 1 \r\n\t\r\n\tfrom ChargeDetail cd\r\n\tinner join Encounter e WITH (NOLOCK) on cd.DateOfService = e.DateOfService and cd.PatientID = e.PatientID\r\n\tleft outer join AuthorizationsPA apa WITH (NOLOCK) on e.PatientID = apa.PatientID and apa.PayerId = e.PayerID  \r\n\tinner join Transactions t WITH (NOLOCK) on cd.ChargeDetailID = t.ChargeDetailID\r\n\tinner join TransactionReason tr WITH (NOLOCK) on tr.ChargeDetailID = t.ChargeDetailID\r\n\tinner join TransactionReasonCode trc WITH (NOLOCK) on trc.TransactionReasonCodeID = tr.TransactionReasonCodeID\r\n\tinner join Patient p WITH (NOLOCK) on p.PatientID = cd.PatientID\r\n\tinner join Department d WITH (NOLOCK) on d.DepartmentID = cd.DepartmentID\r\n\tinner join Provider pp WITH (NOLOCK) on pp.ProviderID = cd.ProviderID\r\n\tinner join Speciality s WITH (NOLOCK) on s.SpecialityID = pp.SpecialityID\r\n\tinner join DiagnosisCode dc WITH (NOLOCK) on dc.DiagnosisCodeID = cd.DiagnosisCodeID1\r\n\tinner join DiagnosisCode dc2 WITH (NOLOCK) on dc2.DiagnosisCodeID = cd.DiagnosisCodeID2\r\n\tinner join CptCode cptc WITH (NOLOCK) on cptc.CPTCodeID = cd.CPTCodeID\r\n\tinner join Transactiontype tt on t.transactiontypeid = tt.TransactionTypeID\r\n\tWHERE\r\n\tcd.DateOfService between '1/01/2020' and '5/31/2020' \r\n\tand cd.Amount >.01 and t.CloseDate is not null\r\n\tand trc.reasonType = 'Denial'  \r\n\tand tt.TransactionTypeCategory = 'Charges'\r\n\tand trc.ReasonCode NOT IN ('19','20','21','22','24','26','27','28','30','31','32')\r\n\r\n) as DND", 'dbtype': 'mssql', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}")
	#sourcePostExecutionHook(sqlserver)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run eligdenautoclassv1Hooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(sqlserver,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "ChargeDetailID", "transformation_label": "String Indexer"}], "feature": "ChargeDetailID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "22", "mean": "", "stddev": "", "min": "17246A3B-4561-4F4A-A688-078711E2799D", "max": "F9640CD4-30FA-4366-A361-D714E737FABE", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ChargeDetailID"}, {"transformationsData": [{"feature_label": "DateofService", "transformation_label": "Extract Date"}], "feature": "DateofService", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "DateofService"}, {"transformationsData": [{"feature_label": "DateOfEntry", "transformation_label": "Extract Date"}], "feature": "DateOfEntry", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "DateOfEntry"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Amount", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "22", "mean": "660.32", "stddev": "1344.95", "min": "45.0", "max": "6000.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Amount"}, {"transformationsData": [{"feature_label": "TransactionDate", "transformation_label": "Extract Date"}], "feature": "TransactionDate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "TransactionDate"}, {"transformationsData": [{"feature_label": "ReasonDate", "transformation_label": "Extract Date"}], "feature": "ReasonDate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "ReasonDate"}, {"transformationsData": [{"feature_label": "CloseDate", "transformation_label": "Extract Date"}], "feature": "CloseDate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "CloseDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PatientBal", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "22", "mean": "-10.97", "stddev": "51.46", "min": "-241.37", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PatientBal"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "InsuranceBal", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "22", "mean": "-2.65", "stddev": "9.28", "min": "-40.59", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "InsuranceBal"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PrimaryInsuranceBal", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "22", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PrimaryInsuranceBal"}, {"transformationsData": [{"feature_label": "CPTCode", "transformation_label": "String Indexer"}], "feature": "CPTCode", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "22", "mean": "88519.53", "stddev": "9697.68", "min": "67028", "max": "J2778PF5", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CPTCode"}, {"transformationsData": [{"feature_label": "CptModifier", "transformation_label": "String Indexer"}], "feature": "CptModifier", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "22", "mean": "19.15", "stddev": "20.31", "min": "0000", "max": "LT", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CptModifier"}, {"transformationsData": [{"feature_label": "pdiagnosis1", "transformation_label": "String Indexer"}], "feature": "pdiagnosis1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "22", "mean": "", "stddev": "", "min": "D31.32", "max": "H43.813", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "pdiagnosis1"}, {"transformationsData": [{"feature_label": "pdiagnosis2", "transformation_label": "String Indexer"}], "feature": "pdiagnosis2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "22", "mean": "", "stddev": "", "min": "D31.31", "max": "Z79.4", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "pdiagnosis2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "IsBillable", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "22", "mean": "1.0", "stddev": "0.0", "min": "1", "max": "1", "missing": "0"}, "updatedLabel": "IsBillable"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Isauthorized", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "22", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "Isauthorized"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredeterminationIndicator", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "22", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "PredeterminationIndicator"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "MedNecessityRequiredIndicator", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "22", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "MedNecessityRequiredIndic..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "22", "mean": "78.05", "stddev": "11.59", "min": "50", "max": "99", "missing": "0"}, "updatedLabel": "Age"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "22", "mean": "", "stddev": "", "min": "F", "max": "M", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "days_to_denial", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "22", "mean": "3.32", "stddev": "174.19", "min": "-439", "max": "236", "missing": "0"}, "updatedLabel": "days_to_denial"}, {"transformationsData": [{"feature_label": "Reasoncode", "transformation_label": "String Indexer"}], "feature": "Reasoncode", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "22", "mean": "44.4", "stddev": "59.72", "min": "16", "max": "N362", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Reasoncode"}, {"transformationsData": [{"feature_label": "ReasonCategory", "transformation_label": "String Indexer"}], "feature": "ReasonCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "22", "mean": "", "stddev": "", "min": "Benefits Max/Exh", "max": "Timely Filing", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ReasonCategory"}, {"transformationsData": [{"feature_label": "ReasonDescription", "transformation_label": "String Indexer"}], "feature": "ReasonDescription", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "22", "mean": "", "stddev": "", "min": "Claim/service lacks information or has submission/billing error(s). Usage: Do not use this code for claims attachment(s)/other documentation. At least one Remark Code must be provided (may be comprised of either the NCPDP Reject Reason Code, or Remittance Advice Remark Code that is not an ALERT.) Refer to the 835 Healthcare Policy Identification Segment (loop 2110 Service Payment Information REF), if present.", "max": "This service/equipment/drug is not covered under the patient\u2019s current benefit plan", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ReasonDescription"}, {"transformationsData": [{"feature_label": "DepartmentCategory", "transformation_label": "String Indexer"}], "feature": "DepartmentCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "22", "mean": "", "stddev": "", "min": "Cardiology", "max": "Pulmonary ", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DepartmentCategory"}, {"transformationsData": [{"feature_label": "SpecialityCategory", "transformation_label": "String Indexer"}], "feature": "SpecialityCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "22", "mean": "", "stddev": "", "min": "Ophtamology", "max": "Ophtamology", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SpecialityCategory"}, {"transformationsData": [{"feature_label": "ProviderID", "transformation_label": "String Indexer"}], "feature": "ProviderID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "22", "mean": "", "stddev": "", "min": "03151B5D-685B-4211-B01F-7BBB1C4A8A49", "max": "CBBD4925-1757-45BC-A853-1C2CA203B9E3", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ProviderID"}, {"transformationsData": [{"feature_label": "PayerId", "transformation_label": "String Indexer"}], "feature": "PayerId", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "22", "mean": "", "stddev": "", "min": "56BFE483-84DF-4FCD-BB0B-4E41BA841A35", "max": "FCE8E295-4222-43DC-9E1E-13222F1045C7", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PayerId"}, {"transformationsData": [{"feature_label": "SecondaryPayerID", "transformation_label": "String Indexer"}], "feature": "SecondaryPayerID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "22", "mean": "", "stddev": "", "min": "0BC878EE-8F48-415E-9233-25A7C28264DB", "max": "EEAC7713-511B-4D72-A6F2-432DD651D92A", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SecondaryPayerID"}, {"transformationsData": [{"feature_label": "TertiaryPayerID", "transformation_label": "String Indexer"}], "feature": "TertiaryPayerID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "22", "mean": "", "stddev": "", "min": "41B329D5-6CFA-464C-9977-0438E0F9B8CE", "max": "FF58D17E-F6AC-4390-999A-AE5378E83883", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "TertiaryPayerID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "label", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "22", "mean": "0.73", "stddev": "0.46", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "label"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run eligdenautoclassv1Hooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(autofe, ["ChargeDetailID_stringindexer", "Amount", "PatientBal", "InsuranceBal", "PrimaryInsuranceBal", "CPTCode_stringindexer", "CptModifier_stringindexer", "pdiagnosis1_stringindexer", "pdiagnosis2_stringindexer", "IsBillable", "Isauthorized", "PredeterminationIndicator", "MedNecessityRequiredIndicator", "Age", "Gender_stringindexer", "days_to_denial", "Reasoncode_stringindexer", "ReasonCategory_stringindexer", "ReasonDescription_stringindexer", "DepartmentCategory_stringindexer", "SpecialityCategory_stringindexer", "ProviderID_stringindexer", "PayerId_stringindexer", "SecondaryPayerID_stringindexer", "TertiaryPayerID_stringindexer", "DateofService_dayofmonth", "DateofService_month", "DateofService_year", "DateOfEntry_dayofmonth", "DateOfEntry_month", "DateOfEntry_year", "TransactionDate_dayofmonth", "TransactionDate_month", "TransactionDate_year", "ReasonDate_dayofmonth", "ReasonDate_month", "ReasonDate_year", "CloseDate_dayofmonth", "CloseDate_month", "CloseDate_year"], "label")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

